In [2]:
import pickle
import nltk
import csv
import string
#nltk.download('popular')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import os
from collections import defaultdict
from os import listdir
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np
from operator import itemgetter
from collections import Counter
mypath='dataset/'

In [ ]:
def getMovieDat():
    
    #documents = [f for f in listdir(mypath) if f.startswith('data_')]
    #documents = [os.listdir(mypath)]
    data={}
    with open('commentsfile1.csv','r', errors = 'ignore') as data_file:
        reader=csv.DictReader(data_file)
        for row in reader:
            if row['video_id'] in data:
                data[row['video_id']].append(row['comment_text'].encode("ascii", errors="ignore").decode())
            else:
                data[row['video_id']] = [row['comment_text']]
    #documents = [f for f in listdir(mypath) if f.endswith('.txt')]
    lmtzr = WordNetLemmatizer()
    docs=[]
    sent_corp=defaultdict(list)
    tokenizer = RegexpTokenizer(r'\w+')
    wrd_freq=Counter()
    for key in tqdm(data):
        lines = ' '
        for e in data[key]:
            lines = lines + e
        sent = sent_tokenize(lines)
        tmp = ''
        for sen in sent:
            tokens=tokenizer.tokenize(sen)
            tagged = nltk.pos_tag(tokens)
            nouns = [word for word,pos in tagged \
                     if (pos == 'NN' or pos == 'NNP' or pos == 'NNS'\
                         or pos == 'NNPS' or pos=='JJ' and len(word)>2)]
            downcased = [x.lower() for x in nouns]
            downcased = [lmtzr.lemmatize(x) for x in downcased]
            downcased = [x for x in downcased if x not in stopwords.words('english')]
            if len(downcased)>1:
                sent_corp[key].append([set(downcased)]+[sen])
                tmp += ' '.join(downcased)
                for x in downcased: wrd_freq[x]+=1
        docs.append(tmp)
        
    temp_docs = []# [docs[i].replace(j,'') for i in docs  for j,v in wrd_freq if j in i and v==1]
    for i, doc in enumerate(docs):
        for j in wrd_freq:
            nj = ' ' + j + ' '
            if nj in doc and wrd_freq[j]==1:
                try:
                    d = temp_docs[i]
                except:
                    d = doc

                temp = d.replace(nj, ' ')
                
                try:
                    temp_docs[i] = temp
                except:
                    temp_docs.append(temp)
                
    tp_wrds=sorted(wrd_freq.items(), key=itemgetter(1), reverse=True)
    return (temp_docs,tp_wrds,sent_corp)
docs,tp_wrds,sent_corp = getMovieDat()

100%|██████████████████████████████████████████████████████████████████████████████| 2266/2266 [28:02<00:00,  1.35it/s]


In [ ]:
# run this block converts words to integers for LDA input
def getMovNAmazonDat(docs):
     
    # change the location of data here
    print("Function called!!")
    vectorizer = CountVectorizer(min_df=1,stop_words='english')
    for d in docs:
        if not (d.strip()):
            docs.remove(d)
    reditposts=docs[:]
    rv_vec=vectorizer.fit_transform(reditposts)
    f_nam=tuple(vectorizer.get_feature_names()) 
    print("Vectorize done!!")
    return(rv_vec,f_nam,docs)

train_d,f_nam,docs = getMovNAmazonDat(docs)
print("Function over!!")

In [ ]:
# train the traditional LDA model
model = lda.LDA(n_topics=100, n_iter=1000, random_state=1)
model.fit(train_d)
topic_word = model.topic_word_

In [ ]:
topic_word.shape

In [ ]:
model.doc_topic_.shape

In [ ]:
import pandas as pd
topic_doc = pd.DataFrame(model.doc_topic_)
maxprob = []
maxprob = topic_doc.apply(lambda x: x.argmax(), axis=1)

In [ ]:
# qry=set(['screen','brightness','display','lcd','reflectiveness'])
qry=set(['logan','apology','phone','parody','artist'])
n_top_words = 100
top_word_list = []
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(f_nam)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    tmp=set(topic_words)
    #if qry.intersection(tmp):
     #   print('Topic {}: {}'.format(i, ' '.join(topic_words)))
    top_word_list.append('Topic {}: {}'.format(i, ' '.join(topic_words)))

In [ ]:
len(top_word_list)

In [ ]:
result=[]
for i in maxprob:
    result.append(top_word_list[i])

In [ ]:
result[1]